### Social Network Graph
The first part of this project is modeling the social network over which we will simulate different events, namely parties, decay of relationships, falling outs, and meeting people.

We determined that the components of the graph are
* People: Each person is represented by a node
* Frendship / connection: Two connected people have an edge between them. The frequency of there being a connected is modeled off of Dunbar's number [(source)](https://www.bbc.com/future/article/20191001-dunbars-number-why-we-can-only-maintain-150-relationships).
* Friendship level: The friendship level of these two people are represented as the weight of the edge. The specific friendship level is chosen using Dunbar's number [(source)](https://www.bbc.com/future/article/20191001-dunbars-number-why-we-can-only-maintain-150-relationships). The specific break downs are:
    * Enemies (??): $<-1$ 
    * People you can recognize (1500): $\frac{0}{2215} \le x < \frac{1500}{2215}$.For most of our calculations, we will considert this range essentially 0.
    * Acquaintances (500): $\frac{1500}{2215} \le x < \frac{2000}{2215}$
    * Meaningful contacts (150): $\frac{2000}{2215} \le x < \frac{2150}{2215}$
    * Friends (50): $\frac{2150}{2215} \le x < \frac{2200}{2215}$
    * Good Friends (15): $\frac{2200}{2215} \le x < \frac{2215}{2215}$

In [ ]:
import numpy as np
import math
from itertools import permutations, combinations


In [2]:
def generate_graph(n: int, p:float):
    # generate the graph parameters
    connections = np.random.uniform(low=0, high=1, size=((n+1)*(n)//2)) > p
    weights = np.random.uniform(low=0, high=1, size=((n+1)*(n)//2))

    # fill out actual graph
    graph = np.zeros((n, n))
    ui = (np.triu_indices(n)) # indices of the upper triangular matrix
    graph[ui] = weights
    graph[ui] = graph[ui] * connections# np.ma.masked_array(graph[ui], mask=connections) 

    # transpose edges for undirected graph
    graph = graph + np.transpose(graph[:, :])

    x = np.arange(n)

    graph[x, x] = 0 
    
    return graph

In [ ]:
graph = generate_graph(4000, 1-2215/4000)
graph

In [40]:
# This function generates the social connections (direct and indirect) between some given person and all other people. In the instance where this person and another person
# are directly connected, the 'social connection' value will likely be equivalent to their current friendship level. Otherwise, indirect connects can have a social connection
# with the given person if they are typically w/in one degree, with a high level of friendship to the intermediary (some friends, and good friends. Occurs about 1% of the time)
# AND the intermediary has a high degree of friendship to the original person. A friend of a friend, so to speak.

def generate_social_connections(graph, person):
    # Build the initial social connections based off the person's direct network
    connectedness = graph[person].copy()
    
    # Note the blacklisted people
    black_list = np.where(connectedness < 0)[0]

    # Create a filter that removes blacklisted people
    mask = np.zeros(connectedness.size, dtype=bool)
    mask[black_list] = True
    mask[person] = True
    mask = np.vstack((mask, mask))
    

    # Apply the filter to connectedness, giving possible guests
    possible = np.ma.array(np.vstack((connectedness, np.array([1] * connectedness.shape[0]))), mask=mask)

        #print("step 1", possible) #testing
    
    # Continue below operations until the social connection value is arbitrarily low (within the lowest catgory)
    while (possible[0] >= 1500/2215).sum() > 0:

            #print("entered loop!") #testing
        
        # Choose the next highest social connection
        next_guest = np.argmax(possible[0])
        guest_distance = possible[1, next_guest]
        connectedness[next_guest] = possible[0, next_guest]

            #print("next guest:", next_guest, "distance:", guest_distance) #testing

        # Update filter to include the next added guest since we do not want to update this value.
        mask[:, next_guest] = True
        possible = np.ma.array(possible, mask=mask)
            #print("masked off next guest", possible) #testing

        
        # Improve social connections if a person is indirectly connected to the person
        updater = possible[0] < connectedness[next_guest] * graph[next_guest] / (guest_distance + 1)**(1/1.9)
            #print("updater", updater) # testing
        possible[1, updater] = guest_distance + 1
        possible[0, updater] = (connectedness[next_guest] * graph[next_guest] / (possible[1])**(1/1.9))[updater]

            #print("updated possible", possible) #testing
    
    return connectedness

In [41]:
test_graph = generate_graph(4000, 1-2215/4000)
connections = generate_social_connections(test_graph, 4)

print("Direct social connections vs. Total Social Connections")
print("Total edge weight:",sum(test_graph[4]), sum(connections))
print("Number of important edges(non-arbitrary connection):", sum(test_graph[4] > 1500/2215), sum(connections > 1500/2215))
print("Total number of edges:", sum(test_graph[4] > 0), sum(connections > 0))

print("\nGranular breakdown of direct and total social connections")
print("Number of acquaintances+:", sum((test_graph[4]) > 1500/2215),sum(connections > 1500/2215))
print("Number of meaningful contacts+:", sum(test_graph[4] > 2000/2215),sum(connections > 2000/2215))
print("Number of friends+:", sum(test_graph[4]>2150/2215), sum(connections > 2150/2215))
print("Number of good friends+:", sum(test_graph[4]>2200/2215),sum(connections > 2200/2215))

Direct social connections vs. Total Social Connections
Total edge weight: 1090.085038492775 1605.5278014182727
Number of important edges(non-arbitrary connection): 704 1673
Total number of edges: 2206 2737

Granular breakdown of direct and total social connections
Number of acquaintances+: 704 1673
Number of meaningful contacts+: 226 226
Number of friends+: 60 60
Number of good friends+: 11 11


In [ ]:
(1- ((1500/2215) * (2)**(1/1.83))) #Rough percentage of occurence

### Invitations and Attending

In [243]:
def determine_host(num):
    # Randomly select a host from the range of people
    
    return np.random.randint(0, num)
    

In [279]:
def hosts_friends(graph, client):
    return np.random.choice(np.where(graph[client] > 1500/2215)[0])

In [216]:
def invite_random(social_connections):
    social_connections *= np.random.normal(loc=0.5, scale=0.33, size=len(social_connections))
    return np.where(social_connections >= (np.random.uniform(low=0.5, high=0.33*(1.75) + 0.5))*2150/2215)[0]

In [6]:
def invite_threshold(graph, host, threshold = 2000/2215):
    return np.where(graph[host]>threshold)[0]

In [283]:
def attending(host, client, invitees, social_connections, base=0.5):
    mask = np.zeros(social_connections.shape[0], dtype=bool)
    mask[invitees] = True

    p_attend = base + 0.5 * (social_connections - 1500/2215)

    mask = mask & (np.random.uniform(0,1, size=len(social_connections)) < p_attend)
    mask[host] = True
    
    mask[client] = True


    return np.where(mask)[0]

In [207]:
graph = generate_graph(4000, 1-2215/4000)
connections = generate_social_connections(graph, 4)
list1 = invite_threshold(graph, 4)
list2 = invite_random(connections)


attending1 = attending(4, list1, connections)
attending2 = attending(4, list2, connections)

In [208]:
print(len(list1), len(attending1))
print(len(list2), len(attending2))

192 83
22 21


### Meeting People at a party

In [8]:
#Probability of meeting someone at the party
# I assume the probability of meeting someone at a party is proportional to the sum of all edge weights a person has
def prob_meeting(graph, person):
    total_weights=np.sum(graph) 
    person_weights = np.sum(graph[person])
    
    return person_weights/total_weights

# As we meet more people, the connection gets stronger and hence the weight increases by a factor j
def update_weight(graph, attendees, j=0.5):
    update_list = list(combinations(attendees, 2))

    for i in update_list:
        a,b = i

        if abs(graph[a,b]) > 0:
            graph[a,b] += j*(1-abs(graph[a,b]))
            graph[b,a]=graph[a,b] # This will keep the symmetry of the edge
        
        else:
            graph[a,b] = 1750/2215
            graph[b,a] = graph[a,b]
    
    return graph

In [143]:
graph = generate_graph(4000, 0.5)
list2 = invite_random(graph, 2)
# attending2 = attending(list2, graph, 2) 
attending2 = [0, 1, 2]
graph[[0,0, 1], [1,2, 2]]

TypeError: invite_random() takes 1 positional argument but 2 were given

In [ ]:
graph = update_weight(graph, attending2)
graph[[0,0, 1], [1,2, 2]]

### Decaying Friendships and Enemies

In [267]:
# This function will occur after a party happens
# Friendships will decay generally if two people were not in the same party together
# A select few random friendships will become enemies

def friendship_decay(graph, party: list[int]):
    # Get every pairing of people at the party
    if len(party) > 1:
        party_pairs = list(permutations(party, 2))
        x, y = zip(*party_pairs)

        # Take note of the current values of the people who went to the party together
        sustained = graph[list(x), list(y)].copy()

    # Decrease friendship levels
    graph[(graph < 1500 /2215) & (graph > 0)] *= 0.95 #0.60 # May need to play with these rates more to make it correct.
    graph[(graph < 2000 / 2215) & (graph >= 1500/2215)] *= 0.95 #0.8
    graph[(graph < 2150 / 2215) & (graph >= 2000/2215)] *= 0.97 #0.95
    graph[(graph < 2200 / 2215) & (graph >= 2150/2215)] *= 0.99
    graph[(graph <= 1) & (graph >= 2200/2215)] *= 0.997

    # Decrease enemy levels
    graph[(graph >= -1) & (graph < -2200/2215)] *= 0.997
    graph[(graph >= -2200/2215) & (graph < -2150/2215)] *= 0.99
    graph[(graph >= -2150 / 2215) & (graph < -2000/2215)] *= 0.95
    graph[(graph >= -2000/2215) & (graph < -1500/2215)] *= 0.8
    graph[(graph >= -1500 /2215) & (graph < 0)] *= 0.60

    # If the value is under 0.1, set to 0.
    graph[(abs(graph) < 0.1)] = 0

    # Reinstate the social connections level for people at the party
    if len(party) > 1:
        graph[list(x), list(y)] = sustained

    return graph

def sudden_enemies(graph, num):
    n = graph.shape[0]
    i = 0
    while i < num:
        nodes = np.random.randint(0, high=n, size=(2))

        if nodes[0] == nodes[1]:
            i = i-1
        else:
            graph[nodes[0], nodes[1]] = -1
            graph[nodes[1], nodes[0]] = -1
        i += 1
    return graph

In [ ]:
tester = generate_graph(4, 1-2215/4000)
tester = sudden_enemies(tester, 2)

tester

In [ ]:
tester = friendship_decay(tester, party=[0,1,2])
tester

In [ ]:
tester = np.ones((4,4)) * 0.9
tester

In [ ]:
tester = friendship_decay(tester, party=[1,2])
tester

### The simulation

In [288]:
# Create network
network = generate_graph(4000, 0.5)#, 1-2215/4000)

# Select targets
temp = np.where(network == 0)
index = np.random.randint(0, temp[0].shape[0])
a = temp[0][index]
b = temp[1][index]

while a == b: # in the case that a and b are the same person,
    index = np.random.randint(0, temp[0].shape[0])
    a = temp[0][index]
    b = temp[1][index]

# Hosting parties
count = 0
time_step = 0
all_hosts = set()

while network[a, b] == 0: #for i in range(10):
    time_step+=1
    host = determine_host(network.shape[0])# hosts_friends(graph, a)#
    all_hosts.add(host)
    print("Unique hosts: ", len(all_hosts))
    social_connections = generate_social_connections(network, host)
    #print("socials", social_connections)

    invitees = invite_random(social_connections)

    party = attending(host, a, invitees, social_connections, 0.75)
    #print("party", party)
    if a in party:
        count+=1

    network = update_weight(network, party)

    if time_step % 7 == 0:
        network = friendship_decay(network, party)
    
    #newtork = sudden_enemies(graph, np.random.randint(0, 10))
    #print(np.sum(network), "hi!")
    print(count)


Unique hosts:  1
1
Unique hosts:  2
2
Unique hosts:  3
3
Unique hosts:  4
4
Unique hosts:  5
5
Unique hosts:  6
6
Unique hosts:  7
7
Unique hosts:  8
8
Unique hosts:  9
9
Unique hosts:  10
10
Unique hosts:  11
11
Unique hosts:  12
12
Unique hosts:  13
13
Unique hosts:  14
14
Unique hosts:  15
15
Unique hosts:  16
16
Unique hosts:  17
17
Unique hosts:  18
18
Unique hosts:  19
19
Unique hosts:  20
20
Unique hosts:  21
21
Unique hosts:  22
22
Unique hosts:  23
23
Unique hosts:  24
24
Unique hosts:  25
25
Unique hosts:  26
26
Unique hosts:  27
27
Unique hosts:  28
28
Unique hosts:  29
29
Unique hosts:  30
30
Unique hosts:  31
31
Unique hosts:  32
32
Unique hosts:  33
33
Unique hosts:  34
34
Unique hosts:  35
35
Unique hosts:  36
36
Unique hosts:  37
37
Unique hosts:  38
38
Unique hosts:  39
39
Unique hosts:  40
40
Unique hosts:  41
41
Unique hosts:  42
42
Unique hosts:  43
43
Unique hosts:  44
44
Unique hosts:  45
45
Unique hosts:  46
46
Unique hosts:  47
47
Unique hosts:  48
48
Unique hos